In [1]:
import os 
import json
import numpy as np
import random
import itertools
import shutil
def merge_config_func(config1,config2):
    merge_config = {}
    merge_succ = True
    for k,v in config1.items():
        if k =="Scene":
            merge_config[k] = config1[k]
        elif k in config2:
            merge_config[k] = [config1[k],config2[k]]
        elif k =="Start":
            pos1 = config1[k][0]
            pos2 = config2[k][0]
            dis = np.linalg.norm(np.array(pos1)-np.array(pos2))
            if dis<=0.5:
                merge_succ = False
        
    if merge_succ:
        return merge_config
    else:
        return None
    
def load_task_config(path):
    with open(path, "r") as f:
        config = json.load(f)
    return config

data_path = "/data1/linmin/EmbodiedAI/resource/datasets/all_task"
scene_dir_list = os.listdir(data_path)
for scene in scene_dir_list:
    scene_path = os.path.join(data_path, scene)
    sample_List = os.listdir(scene_path)
    valid_task_dirs = [(path,os.path.join(scene_path, path, "config.json")) for id, path in enumerate(sample_List)]
    if len(sample_List)>=2:
        for (id1, path1), (id2, path2) in itertools.combinations(valid_task_dirs, 2):
            config1 = load_task_config(path1)
            config2 = load_task_config(path2)
            process_config1 = load_task_config(path1.replace("config","processed_config"))
            process_config2 = load_task_config(path2.replace("config","processed_config"))
            merge_config = merge_config_func(config1,config2)
            merge_process_config = merge_config_func(process_config1,process_config2)
            if merge_config:
                if not os.path.exists(os.path.dirname(path1.replace("all_task","merge_task").replace("/"+id1+"/","/"+id1+"_"+id2+"/"))):
                    os.makedirs(os.path.dirname(path1.replace("all_task","merge_task").replace("/"+id1+"/","/"+id1+"_"+id2+"/")))
                with open(path1.replace("all_task","merge_task").replace("/"+id1+"/","/"+id1+"_"+id2+"/"), "w") as f:
                    json.dump(merge_config, f, indent=4)
                with open(path1.replace("all_task","merge_task").replace("/"+id1+"/","/"+id1+"_"+id2+"/").replace("config","processed_config"), "w") as f:
                    json.dump(merge_process_config, f, indent=4)
                img_path1 = path1.replace("config.json","final_image.png")
                img2_path1 = path1.replace("config.json","final_image2.png")
                img_path2 = path2.replace("config.json","final_image.png")
                img2_path2 = path2.replace("config.json","final_image2.png")
                shutil.copy(img_path1, img_path1.replace("all_task","merge_task").replace("/"+id1+"/","/"+id1+"_"+id2+"/").replace("final_image","final_image1_1"))
                shutil.copy(img2_path1, img2_path1.replace("all_task","merge_task").replace("/"+id1+"/","/"+id1+"_"+id2+"/").replace("final_image2","final_image1_2"))
                shutil.copy(img_path2, img_path2.replace("all_task","merge_task").replace("/"+id2+"/","/"+id1+"_"+id2+"/").replace("final_image","final_image2_1"))
                shutil.copy(img2_path2, img2_path2.replace("all_task","merge_task").replace("/"+id2+"/","/"+id1+"_"+id2+"/").replace("final_image2","final_image2_2"))
                # shutils.copy(path1, path1.replace("all_task","merge_task"))
        
